In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np  # Importa numpy per la gestione delle trasformazioni

# Percorsi dei file Excel
f1 = "centrocampisti_2020.xlsx"
f2 = "centrocampisti_2021.xlsx"
f3 = "centrocampisti_2022.xlsx"
f4 = "centrocampisti_2023.xlsx"
f5 = "centrocampisti_2024.xlsx"

# Colonne da eliminare
colonne_da_eliminare = ["Player", "Nation", "Pos", "Squad", "Age", "MP", "Starts", "Min", "G + A", "G -PK","Assist",
                        "PK", "PKA", "YCrd", "RCrd", "xG", "xG-PK", "Xg kPass", "npxG+xAG", "PrgC", "PrgP", "PrgPr", "Goals",
                        "Shot", "SoT", "%SoT", "PK", "PKA", "xG", "X-XG", "%Pass", "D(yrd)", "PrgDist", "PoT1",
                        "xPoT1", "%PoT", "PoT2", "xPoT2", "%PoT2", "PoT3", "xPoT3", "%PoT3", "Ass", "xAss", "PPA",
                        "CPA", "PrgP", "Fkpass", "PassDefL", "CdC", "Cross", "GoalActionsAtt", "GoalAction",
                        "Att", "Succ", "Succ%", "DistPA", "2CrdY","Cross"]

# Leggi e unisci i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Rimuovi le colonne specificate dal DataFrame unito
df_unito = df_unito.drop(columns=colonne_da_eliminare, errors='ignore')

# Carica il file centrocampisti_23.xlsx e rimuovi le stesse colonne
df5 = pd.read_excel(f5).iloc[1:].reset_index(drop=True)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')

# Separare le colonne in input e output
X = df_unito.drop(columns=['KeyP','interc_pass'])
y = df_unito[['KeyP','interc_pass']] 

# Pre-elaborazione
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Costruzione della rete neurale
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1])
])

# Compilazione del modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Addestramento del modello
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Carica il nuovo file e pre-processa i dati
df_new = df5  # Usa il DataFrame df5 che abbiamo già elaborato

# Verifica che le colonne siano allineate con il DataFrame di addestramento
X_new = df_new[X.columns]

# Pre-elabora i dati nuovi (normalizza con lo stesso scaler usato per l'addestramento)
X_new_scaled = scaler.transform(X_new)

# Usa il modello per fare previsioni
y_pred = model.predict(X_new_scaled)

# Imposta a 0 tutti i valori predetti che sono minori di 0
y_pred = np.clip(y_pred, 0, None)  # Sostituisci i valori negativi con 0

# Visualizza le previsioni
df_new['Pred_KeyP'] = y_pred[:, 0]  # Previsioni per la prima colonna (Gls)
df_new['Pred_intercBall'] = y_pred[:, 1]  # Previsioni per la seconda colonna (Ast)

print(df_new[['KeyP','interc_pass','Pred_KeyP','Pred_intercBall']])





# Calcola separatamente la somma delle previsioni per Gls e Ast
somma_Kp_Attesi = df_new['Pred_KeyP'].sum()
somma_Ib_Attesi = df_new['Pred_intercBall'].sum()

# Visualizza la somma
print(f"La somma dei passaggi chiave ottenuti dalla rete neurale è: {somma_Kp_Attesi}")
print(f"La somma delle palle intercettate ottenuti dalla rete neurale è: {somma_Ib_Attesi}")
somma_tot= somma_Kp_Attesi + somma_Ib_Attesi

# Visualizza la somma
print(f"La somma dei passaggi chiave e passaggi intercettati ottenuti dalla rete neurale è: {somma_tot}")

# Calcola separatamente la somma effettiva dei goal e assist nei dati originali
somma_Kp_effettiva = df_new['KeyP'].sum()
somma_Ib_effettiva = df_new['interc_pass'].sum()

somma_effettiva = somma_Kp_effettiva + somma_Ib_effettiva

print(f"La somma dei passaggi chiave e passaggi intercettati effettiva è: {somma_effettiva}")

Epoch 1/100


C:\Users\Mario\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\Mario\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\Mario\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 420.1244 - val_loss: 406.9243
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 423.4367 - val_loss: 352.1638
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 347.6890 - val_loss: 270.1066
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 291.1382 - val_loss: 163.6756
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 181.8684 - val_loss: 107.0133
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 116.2427 - val_loss: 101.0933
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 105.7267 - val_loss: 91.1466
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 104.2930 - val_loss: 84.1862
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 93.7839 - val_loss: 76.5447
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 89.5917 - val_loss: 70.1256
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 74.1137 - val_loss: 64.6702
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.6316 - val_loss: 37.6194
Epoch 69/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.1174 - val_loss: 37.6420
Epoch 70/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.1118 - val_loss: 37.2526
Epoch 71/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.7210 - val_loss: 37.3510
Epoch 72/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 33.4786 - val_loss: 37.1345
Epoch 73/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32.9234 - val_loss: 37.5411
Epoch 74/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.1414 - val_loss: 36.9376
Epoch 75/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27.1064 - val_loss: 37.3278
Epoch 76/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27.1933 - val_loss: 36.7419
Epoch 77/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26.3281 - val_loss: 36.9066
Epoch 78/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.0153 - val_loss: 36.8931
Epoch 79/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4m

In [2]:
# Errore

diff_Kp = abs(somma_Kp_effettiva - somma_Kp_Attesi)
diff_Ib = abs(somma_Ib_effettiva - somma_Ib_Attesi)

# Trovo l'errore medio dividendo per il numero di giocatori 

avg_err_Kp = diff_Kp/154  
avg_err_Ib = diff_Ib/154

print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_Kp} Gol")
print(f"La differenza fra la somma effettiva e quella della rete neurale è di {diff_Ib} Assist")
print(f"L'errore medio per i passaggi chiave è {avg_err_Kp} per giocatore")
print(f"L'errore medio per le palle intercettate è {avg_err_Ib} per giocatore")

La differenza fra la somma effettiva e quella della rete neurale è di 611.42578125 Gol
La differenza fra la somma effettiva e quella della rete neurale è di 156.2108154296875 Assist
L'errore medio per i passaggi chiave è 3.970297280844156 per giocatore
L'errore medio per le palle intercettate è 1.01435594434862 per giocatore
